## LLM、Prompt

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0)
chat.predict_messages([HumanMessage(content="把下面的语言翻译为中文：I love programming.")])


AIMessage(content='我喜欢编程。', additional_kwargs={}, example=False)

## LLM、PromptTemplate、Chain

In [2]:
from langchain import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate.from_template("帮我为以下产品想一个公司的名称 {product}?")
chain = LLMChain(llm=chat, prompt=prompt)
chain.run("袜子")

'1. 脚云袜子 (FootCloud Socks)\n2. 绵绵袜子 (SoftCotton Socks)\n3. 舒适足袜 (ComfortFit Socks)\n4. 时尚足袜 (FashionFoot Socks)\n5. 柔软足袜 (GentleTouch Socks)\n6. 足底护理袜 (SoleCare Socks)\n7. 活力足袜 (VitalityFoot Socks)\n8. 温暖足袜 (WarmFeet Socks)\n9. 亲肤足袜 (SkinFriendly Socks)\n10. 美丽足袜 (BeautifulFeet Socks)'

## Memory,Chain

In [13]:
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain, PromptTemplate


In [14]:
template = """你是一个和人类聊天的机器人.

{chat_history}
人类: {human_input}
机器人:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")
llm_chain = LLMChain(
    llm=OpenAI(),
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [15]:
llm_chain.predict(human_input="你好，我的名字叫致问")



> Entering new LLMChain chain...
Prompt after formatting:
你是一个和人类聊天的机器人.


人类: 你好，我的名字叫致问
机器人:

> Finished chain.


' 你好，致问，很高兴认识你！有什么可以帮助你的吗？'

In [16]:
memory.buffer

'Human: 你好，我的名字叫致问\nAI:  你好，致问，很高兴认识你！有什么可以帮助你的吗？'

In [17]:
llm_chain.predict(human_input="我的名字叫什么？")



> Entering new LLMChain chain...
Prompt after formatting:
你是一个和人类聊天的机器人.

Human: 你好，我的名字叫致问
AI:  你好，致问，很高兴认识你！有什么可以帮助你的吗？
人类: 我的名字叫什么？
机器人:

> Finished chain.


' 哦，你是叫致问吗？'

## Agent、LLM、Prompt、Memory

In [5]:
from langchain.memory import ConversationBufferMemory
from langchain.agents import load_tools, AgentExecutor, ZeroShotAgent
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI



## 1、定义工具
tools = load_tools(["serpapi"])

prefix = """与人进行对话，尽可能最好地回答以下问题。你可以使用以下工具："""
suffix = """开始！

{chat_history}
问题：{input}
{agent_scratchpad}"""

## 2、定义提示词
prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)

## 定义链和Agent
memory = ConversationBufferMemory(memory_key="chat_history")
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)


In [7]:
agent_chain.run(input="今天杭州的天气怎么样?")



> Entering new AgentExecutor chain...
Thought: 我需要查询一下杭州的天气。
Action: Search
Action Input: 杭州天气
Observation: 20日（今天）. 多云. 26℃. <3级 · 21日（明天）. 雷阵雨. 33℃/25℃. 4-5级转<3级 · 22日（后天）. 雷阵雨. 34℃/26℃. 5-6级转3-4级 · 23日（周日）. 雷阵雨转多云. 33℃/26℃. 4-5 ...
Thought: 我现在知道最终的答案了。
Final Answer: 今天杭州的天气是多云，温度为26℃，风力为3级。

> Finished chain.


'今天杭州的天气是多云，温度为26℃，风力为3级。'

In [9]:
memory.buffer

'Human: 今天杭州的天气怎么样?\nAI: 今天杭州的天气是多云，温度为26℃，风力为3级。'

In [10]:
agent_chain.run(input="能解释下LangChain是什么吗，开发时间和开发背景?")



> Entering new AgentExecutor chain...
Thought: 我需要搜索LangChain的信息
Action: Search
Action Input: LangChain
Observation: LangChain is a framework designed to simplify the creation of applications using large language models. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.
Thought: 我现在了解了LangChain的开发背景
Final Answer: LangChain是一个旨在简化使用大型语言模型创建应用程序的框架。作为一个语言模型集成框架，LangChain的用例与一般的语言模型大部分重叠，包括文档分析和摘要、聊天机器人和代码分析。

> Finished chain.


'LangChain是一个旨在简化使用大型语言模型创建应用程序的框架。作为一个语言模型集成框架，LangChain的用例与一般的语言模型大部分重叠，包括文档分析和摘要、聊天机器人和代码分析。'

In [11]:
memory.buffer

'Human: 今天杭州的天气怎么样?\nAI: 今天杭州的天气是多云，温度为26℃，风力为3级。\nHuman: 能解释下LangChain是什么吗，开发时间和开发背景?\nAI: LangChain是一个旨在简化使用大型语言模型创建应用程序的框架。作为一个语言模型集成框架，LangChain的用例与一般的语言模型大部分重叠，包括文档分析和摘要、聊天机器人和代码分析。'